# 🚀 YouTube Viral Clipper - Google Colab (GPU T4 Compatible)

Aplikasi ini akan mengubah video YouTube menjadi klip viral dengan AI.

**✨ Fitur:**
- 🤖 AI-Powered Clip Selection (Gemini API)
- 🎯 Smart Face Tracking (MediaPipe)
- 📝 Word-by-Word Captions (Whisper AI)
- ⚡ GPU Accelerated (T4 Compatible)

---

## 📋 Cara Penggunaan:
1. Jalankan **Cell 1** untuk setup (hanya sekali)
2. Jalankan **Cell 2** untuk generate video (bisa diulang)


---

## 📦 CELL 1: Setup & Instalasi

**Jalankan cell ini SEKALI untuk install semua dependencies.**

Proses ini memakan waktu 2-3 menit.


In [ ]:
import os
import sys
import subprocess
from pathlib import Path
from getpass import getpass

print("🚀 YouTube Viral Clipper - Smart Setup")
print("=" * 60)

# ============================================================================
# STEP 1: Install Dependencies (Smart Check)
# ============================================================================
print("\n📦 Step 1: Installing dependencies...")
print("   (This may take 2-3 minutes)\n")

# Check if already installed
deps_installed = False
try:
    import dotenv, pytubefix, moviepy, faster_whisper, torch, google.generativeai, cv2, numpy, PIL
    print("   ✅ Main dependencies already installed")
    deps_installed = True
except ImportError:
    print("   ⏳ Installing main dependencies...")
    deps_installed = False

if not deps_installed:
    !pip install -q python-dotenv pytubefix moviepy google-generativeai pillow yt-dlp
    !pip install -q faster-whisper
    !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    !pip install -q opencv-python
    print("   ✅ Main dependencies installed!")

# ============================================================================
# STEP 2: Verify OpenCV (for face tracking)
# ============================================================================
print("\n🎯 Step 2: Verifying OpenCV for face tracking...")

# OpenCV should already be installed, just verify
try:
    import cv2
    # Test Haar Cascade availability
    cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    test_cascade = cv2.CascadeClassifier(cascade_path)
    if test_cascade.empty():
        raise RuntimeError("Haar Cascade not found")
    print("   ✅ OpenCV face detection ready!")
    print(f"   📌 OpenCV Version: {cv2.__version__}")
    print("   💡 Using Haar Cascade (no MediaPipe issues!)")
except Exception as e:
    print(f"   ⚠️  OpenCV test failed: {e}")
    print("   🔄 Please RESTART RUNTIME and run this cell again.")
    raise

# ============================================================================
# STEP 3: Setup Code (Clone or Upload)
# ============================================================================
print("\n📂 Step 3: Setting up code...")

repo_url = "https://github.com/portdigital/testing-AI-Clipping-Software.git"
project_dir = "/content/AI-Clipping-Software"

if not os.path.exists(project_dir):
    print(f"   ⏳ Cloning repository from GitHub...")
    result = subprocess.run(
        ["git", "clone", repo_url, project_dir],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print("   ✅ Repository cloned successfully!")
    else:
        print(f"   ❌ Clone failed: {result.stderr}")
        raise RuntimeError("Failed to clone repository")
else:
    print("   ✅ Code already exists")

# Change to project directory
os.chdir(project_dir)
sys.path.insert(0, project_dir)
print(f"   📁 Working directory: {os.getcwd()}")

# Create output directories
os.makedirs("output", exist_ok=True)
os.makedirs("temp", exist_ok=True)
print("   ✅ Output directories created")

# ============================================================================
# STEP 4: Setup Gemini API Key
# ============================================================================
print("\n🔑 Step 4: Setting up Gemini API Key...")

# Check if .env exists
env_file = Path(".env")
if env_file.exists():
    print("   ✅ API key already configured")
else:
    print("   ⚠️  No API key found. Please enter your Gemini API key.")
    print("   💡 Get your free API key: https://aistudio.google.com/app/apikey")
    api_key = getpass("   🔑 Enter your Gemini API Key: ")
    
    if api_key:
        with open(".env", "w") as f:
            f.write(f"GEMINI_API_KEY={api_key}\n")
        print("   ✅ API key saved!")
    else:
        print("   ⚠️  No API key entered. You'll need to add it manually later.")

# ============================================================================
# STEP 5: Test Imports
# ============================================================================
print("\n🧪 Step 5: Testing imports...")

try:
    from services.video_processor import VideoProcessor
    from styles.caption_styles import CAPTION_STYLES
    from config import GEMINI_API_KEY
    print("   ✅ All imports successful!")
except Exception as e:
    print(f"   ❌ Import error: {e}")
    raise

# ============================================================================
# STEP 6: Check GPU
# ============================================================================
print("\n🖥️  Step 6: Checking GPU...")

try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"   ✅ GPU Available: {gpu_name}")
        print(f"   📌 CUDA Version: {torch.version.cuda}")
        print(f"   📌 PyTorch Version: {torch.__version__}")
    else:
        print("   ⚠️  No GPU detected. Enable GPU: Runtime → Change runtime type → T4 GPU")
except Exception as e:
    print(f"   ⚠️  GPU check failed: {e}")

# ============================================================================
# DONE!
# ============================================================================
print("\n" + "=" * 60)
print("✅ SETUP COMPLETE! Ready to generate viral clips!")
print("=" * 60)
print("\n📌 Next step: Run CELL 2 to generate your video clips!\n")


---

## 🎬 CELL 2: Generate Video Clips

**Jalankan cell ini untuk membuat klip viral dari YouTube video.**

Anda bisa menjalankan cell ini berulang kali dengan video yang berbeda.


In [ ]:
import os
import sys
from pathlib import Path
from google.colab import files
import zipfile
import shutil

# Ensure we're in the project directory
project_dir = "/content/AI-Clipping-Software"
if os.getcwd() != project_dir:
    os.chdir(project_dir)
    sys.path.insert(0, project_dir)

# Import modules
from services.video_processor import VideoProcessor
from styles.caption_styles import CAPTION_STYLES

print("🎬 YouTube Viral Clipper - Video Generator")
print("=" * 60)

# ============================================================================
# USER INPUT
# ============================================================================
print("\n📝 Step 1: Enter video details...\n")

# YouTube URL
youtube_url = input("🔗 YouTube URL: ").strip()
if not youtube_url:
    print("❌ Error: YouTube URL is required!")
    raise ValueError("YouTube URL cannot be empty")

# Number of clips
num_clips_input = input("🎯 Number of clips (default: 3): ").strip()
num_clips = int(num_clips_input) if num_clips_input else 3

# Minimum duration
min_duration_input = input("⏱️  Min duration in seconds (default: 30): ").strip()
min_duration = int(min_duration_input) if min_duration_input else 30

# Maximum duration
max_duration_input = input("⏱️  Max duration in seconds (default: 60): ").strip()
max_duration = int(max_duration_input) if max_duration_input else 60

# Caption style
print("\n🎨 Available caption styles:")
for idx, (style_name, style_config) in enumerate(CAPTION_STYLES.items(), 1):
    print(f"   {idx}. {style_name} - {style_config.get('description', 'No description')}")

style_choice_input = input(f"\n🎨 Choose caption style (1-{len(CAPTION_STYLES)}, default: 1): ").strip()
style_choice = int(style_choice_input) if style_choice_input else 1

style_names = list(CAPTION_STYLES.keys())
if 1 <= style_choice <= len(style_names):
    selected_style = style_names[style_choice - 1]
else:
    print("⚠️  Invalid choice, using default style (Clean White)")
    selected_style = "clean_white"

print(f"\n✅ Selected style: {selected_style}")

# ============================================================================
# PROCESS VIDEO
# ============================================================================
print("\n" + "=" * 60)
print("🚀 Starting video processing...")
print("=" * 60)

try:
    processor = VideoProcessor(caption_style=selected_style)
    outputs, title = processor.process_video(
        url=youtube_url,
        num_clips=num_clips,
        min_duration=min_duration,
        max_duration=max_duration
    )
    
    print("\n" + "=" * 60)
    print("✅ VIDEO PROCESSING COMPLETE!")
    print("=" * 60)
    print(f"\n📹 Generated {len(outputs)} clips from: {title}")
    
    # ============================================================================
    # DOWNLOAD RESULTS
    # ============================================================================
    print("\n📦 Preparing download...")
    
    # Create zip file
    output_dir = Path("output")
    zip_filename = f"viral_clips_{title[:30].replace(' ', '_')}.zip"
    zip_path = output_dir / zip_filename
    
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for output_file in outputs:
            if os.path.exists(output_file):
                zipf.write(output_file, os.path.basename(output_file))
    
    print(f"✅ Created zip file: {zip_filename}")
    print(f"📦 Size: {os.path.getsize(zip_path) / (1024*1024):.2f} MB")
    
    # Download
    print("\n⬇️  Downloading...")
    files.download(str(zip_path))
    print("✅ Download started! Check your browser's download folder.")
    
    # Cleanup
    print("\n🧹 Cleaning up temporary files...")
    shutil.rmtree("temp", ignore_errors=True)
    os.makedirs("temp", exist_ok=True)
    print("✅ Cleanup complete!")
    
    print("\n" + "=" * 60)
    print("🎉 ALL DONE! Your clips are ready!")
    print("=" * 60)
    print("\n💡 Want to process another video? Just run this cell again!\n")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    print("\n💡 Tips:")
    print("   - Check your YouTube URL")
    print("   - Verify your Gemini API key is correct")
    print("   - Make sure GPU is enabled (Runtime → Change runtime type → T4 GPU)")
    print("   - Try running CELL 1 again to reset everything")
    raise
